p_02 | `BeautifulSoup`
====

Ejercicio 1
----

1. De [esta URL](https://autos.mercadolibre.com.uy/usado/dueno/2016-2019/auto_NoIndex_True_VEHICLE*BODY*TYPE_479344) (solo la primera página) extraer la siguiente información para cada publicación y construir un DataFrame. Agregar una columna con la URL de la publicación correspondiente.
    1. Nombre (Marca, modelo, etc.)
    2. Año
    3. Kilometraje
    4. Ubicación
    5. Precio
2. Descargar la imagen correspondiente a cada publicación (solo la que aparece en la búsqueda). El nombre del archivo debe referenciar la publicación (ya sea por marca-modelo o por un identificador común). No sobreescribir la `request` original.
3. Ingresar a cada publicación y crear un DataFrame a partir de la tabla de "Características principales" (reutilizar la `request` original). Cada una de las características debe ser una columna, y cada fila un auto. Concatenar este DataFrame con el original, haciendo las modificaciones necesarias para que no haya columnas repetidas.

In [6]:
# Ejercicio 1 | Solución 1

import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://autos.mercadolibre.com.uy/usado/dueno/2016-2019/auto_NoIndex_True_VEHICLE*BODY*TYPE_479344"
r = requests.get(base_url)
soup = BeautifulSoup(r.text)
urls = [u.get("href") for u in soup.find_all("a", class_="ui-search-result__content ui-search-link")]
titles = [t.text for t in soup.find_all("h2", class_="ui-search-item__title ui-search-item__group__element")]
attributes = soup.find_all("ul", class_="ui-search-card-attributes ui-search-item__group__element")
years = [a.contents[0].text for a in attributes]
kms = [a.contents[1].text for a in attributes]
prices = [s.text for s in soup.find_all("span", class_="price-tag-fraction")]
locations = [l.text for l in soup.find_all("span", class_="ui-search-item__group__element ui-search-item__location")]
data = {"Auto": titles, "Año": years, "Kilometraje": kms, "Precio": prices[1:], "Ubicación": locations, "URL": urls}
df = pd.DataFrame(data)
df.head()


,Auto,Año,Kilometraje,Precio,Ubicación,URL
0,Chevrolet Cruze 5 1.4 Ltz Plus 153cv,2017,147.000 Km,22.500,Rivera - Rivera,https://auto.mercadolibre.com.uy/MLU-479553723...
1,Volkswagen Gol 1.6 Power 101cv,2018,35.000 Km,13.800,Malvin - Montevideo,https://auto.mercadolibre.com.uy/MLU-479540109...
2,Mini Cooper Countryman 2.0 S 192cv,2018,47.000 Km,41.900,Carrasco - Montevideo,https://auto.mercadolibre.com.uy/MLU-479316295...
3,Suzuki Swift 1.2 Go,2018,70.000 Km,11.700,Cerrito - Montevideo,https://auto.mercadolibre.com.uy/MLU-478794828...
4,Geely Lc 1.0 Gl Full Gc2,2017,87.750 Km,7.300,Malvin - Montevideo,https://auto.mercadolibre.com.uy/MLU-479306530...


In [2]:
# Ejercicio 1 | Solución 2
from pathlib import Path

img_urls = [img.get("data-src") for img in soup.find_all("img", class_="ui-search-result-image__element")]
filenames = df["Auto"].to_list()

with requests.Session() as s:
    for url, filename in zip(img_urls, filenames):
        r_img = s.get(url)
        with open(f"{filename}.jpg", "wb") as f:
            f.write(r_img.content)

[x.name for x in Path(".").glob("*.jpg")]

['Audi A3 1.2 Tfsi Attraction 105 Hp 5p.jpg',
 'Audi S3 2.0 Tfsi Stronic Quattro 300cv.jpg',
 'Chevrolet Cruze 5 1.4 Ltz Plus 153cv.jpg',
 'Chevrolet Onix 1.0 Joy 78cv.jpg',
 'Chevrolet Onix 1.0 Lt 78cv.jpg',
 'Chevrolet Onix 1.4 Activ.jpg',
 'Chevrolet Onix Joy 1.0 Financio100%.jpg',
 'Citroën C3 1.2 Puretech 82 Feel Europa.jpg',
 'Citroën C3 1.2 Puretech 82 Shine Europa.jpg',
 'Citroën C3 Aircross 1.2 Pure Tech 110 5v Feel Europa.jpg',
 'Citroën Ds3 1,2.jpg',
 'Fiat 500l 1.4 Pop Star 105cv.jpg',
 'Fiat Mobi 1.0 Easy On.jpg',
 'Fiat Uno 1.4 Way Lx.jpg',
 'Ford Fiesta S Plus.jpg',
 'Geely Lc 1.0 Gl Full Gc2.jpg',
 'Honda Civic 1.5 Ex-t 2017.jpg',
 'Honda Fit 1.5 Ex-l At 120cv.jpg',
 'Kia Picanto 1.0 Ex Plus 5p.jpg',
 'Kia Picanto 1.2 Ex 85cv 5mt.jpg',
 'Kia Sportage 2.0 Ex 2wd 163cv Mt.jpg',
 'Mini Cooper Countryman 2.0 S 192cv.jpg',
 'Nissan March 1.6 Advance At.jpg',
 'Nissan March 1.6 Advance Mt.jpg',
 'Peugeot 208 1.0 Access 5p.jpg',
 'Peugeot 208 1.6 Allure.jpg',
 'Peugeot 308 1.6

In [4]:
# Ejercicio 1 | Solución 3

tables = []
with requests.Session() as s:
    for url in urls:
        r_post = s.get(url)
        table = pd.read_html(r_post.text, attrs={"class": "andes-table"}, flavor="html5lib", index_col=0)[0]
        tables.append(table)
full_table = pd.concat(tables, axis=1).T.reset_index(drop=True).drop(["Año", "Kilómetros"], axis=1)
final = pd.concat([df, full_table], axis=1)
final.head()


,Auto,Año,Kilometraje,Ubicación,URLs,Marca,Modelo,Color,Tipo de combustible,Puertas,Transmisión,Motor,Tipo de carrocería
0,Chevrolet Cruze 5 1.4 Ltz Plus 153cv,2017,147.000 Km,Rivera - Rivera,https://auto.mercadolibre.com.uy/MLU-479553723...,Chevrolet,Cruze 5,Plateado,Nafta,5,Automática,1.4,Hatchback
1,Volkswagen Gol 1.6 Power 101cv,2018,35.000 Km,Malvin - Montevideo,https://auto.mercadolibre.com.uy/MLU-479540109...,Volkswagen,Gol,Plateado,Nafta,5,Manual,1.6,Hatchback
2,Mini Cooper Countryman 2.0 S 192cv,2018,47.000 Km,Carrasco - Montevideo,https://auto.mercadolibre.com.uy/MLU-479316295...,Mini,Cooper Countryman,Blanco,Nafta,5,Manual,2.0,Hatchback
3,Suzuki Swift 1.2 Go,2018,70.000 Km,Cerrito - Montevideo,https://auto.mercadolibre.com.uy/MLU-478794828...,Suzuki,Swift,Plateado,Nafta,5,Manual,1.2,Hatchback
4,Geely Lc 1.0 Gl Full Gc2,2017,87.750 Km,Malvin - Montevideo,https://auto.mercadolibre.com.uy/MLU-479306530...,Geely,LC 1.0 GL FULL,Blanco,Nafta,5,Manual,1.000 cm3,Hatchback
